In [13]:
import numpy as np
np.random.seed(42)
import pandas as pd
# from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from surprise import AlgoBase, PredictionImpossible, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split
from surprise import SVDpp

all_df = pd.read_csv("../../data/data_train.csv")
all_df['user_id'] = all_df['Id'].apply(lambda x: int(x.split("_")[0].split("r")[-1]))
all_df['movie_id'] = all_df['Id'].apply(lambda x: int(x.split("_")[-1].split("c")[-1]))
all_df['rating'] = all_df['Prediction'].values
all_df = all_df[['user_id', 'movie_id', 'rating']]
number_of_users = all_df.user_id.unique().shape[0]
number_of_movies = all_df.movie_id.unique().shape[0]

# train_df, test_df = train_test_split(all_df, test_size=0.1, random_state=42)

reader = Reader()
dataset = Dataset.load_from_df(all_df, reader)

trainset, testset = train_test_split(dataset, test_size=0.10)

model = SVDpp(verbose=True, n_factors=192, n_epochs=85)

# fit the model to the training set
model.fit(trainset)

# compute the predictions
predictions = model.test(testset)

# compute the RMSE on the testset
rmse = accuracy.rmse(predictions)

# data = np.full((number_of_users, number_of_movies), np.mean(train_df.rating.values))
# mask = np.zeros((number_of_users, number_of_movies))
# for user_id, movie_id, rating in zip(train_df.user_id.tolist(), train_df.movie_id.tolist(), train_df.rating.tolist()):
#     data[user_id - 1][movie_id - 1] = rating
#     mask[user_id - 1][movie_id - 1] = 1

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
 processing epoch 20
 processing epoch 21
 processing epoch 22
 processing epoch 23
 processing epoch 24
 processing epoch 25
 processing epoch 26
 processing epoch 27
 processing epoch 28
 processing epoch 29
 processing epoch 30
 processing epoch 31
 processing epoch 32
 processing epoch 33
 processing epoch 34
 processing epoch 35
 processing epoch 36
 processing epoch 37
 processing epoch 38
 processing epoch 39
 processing epoch 40
 processing epoch 41
 processing epoch 42
 processing epoch 43
 processing epoch 44
 processing epoch 45
 processing epoch 46
 processing epoch 47
 p

In [14]:
rmse

1.0779551864562102

In [2]:
def extract_prediction_from_full_matrix(reconstructed_matrix):
    predictions = np.zeros(test_df.shape[0])
    for i, (user_id, movie_id) in enumerate(zip(test_df.user_id.tolist(), test_df.movie_id.tolist())):
        predictions[i] = reconstructed_matrix[user_id - 1][movie_id - 1]
    return predictions

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [3]:
k_singular_values = 10
number_of_singular_values = min(number_of_users, number_of_movies)

U, s, Vt = np.linalg.svd(data, full_matrices=False)

S = np.zeros((number_of_movies, number_of_movies))
S[:k_singular_values, :k_singular_values] = np.diag(s[:k_singular_values])

reconstructed_matrix = U.dot(S).dot(Vt)

predictions = extract_prediction_from_full_matrix(reconstructed_matrix)

print("RMSE using SVD is: {:.4f}".format(rmse(test_df.rating.values, predictions)))

RMSE using SVD is: 1.0657


In [3]:
#@title Use GPU in colab: Runtime->Change Runtime type
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
# from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm

#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using device:', device)

Using device: cpu


In [4]:
class NMF(nn.Module):

    def __init__(self):
        super().__init__()
        self.user_dim = 10000
        self.item_dim = 1000
        
        self.layer_1 = nn.Linear(1000, 100)
        self.layer_1a = nn.Linear(100, 100)
        
        # self.vae = nn.Linear(100, 100)
        
        # self.layer_3 = nn.Linear(50, 100)
        self.cls_layer = nn.Linear(100, 1000)

    def forward(self, data):
        net_data = data

        # Encoder layers
        net_data = F.relu(self.layer_1(net_data))
        net_data = F.dropout(net_data, training=self.training)
        
        net_data = F.relu(self.layer_1a(net_data))
        net_data = F.dropout(net_data, training=self.training)
        
        # VAE bottleneck
        # vae = self.vae(net_data)
        # mus_q, log_sigmas_q = torch.split(vae, 50, dim=-1)
        # stds_q = torch.exp(0.5 * log_sigmas_q)
        
        # KL = 0.5 * (-log_sigmas_q + torch.exp(log_sigmas_q) + mus_q ** 2 - 1)
        # KL = torch.sum(KL, dim=-1)
        # KL = torch.mean(KL)
        
        # Sample random value if training,
        # use the mean during evaluation
        # if not self.training:
        #     sampled_z = mus_q
        # else:
        #     eps = torch.randn(stds_q.shape, dtype=torch.float, device=device)
        #     sampled_z = mus_q + eps * stds_q
        
        # Decoder layers
        # net_data = F.relu(self.layer_3(sampled_z))
        # net_data = F.dropout(net_data, training=self.training)
        
        # Classification layer
        y_score = self.cls_layer(net_data)
        return y_score

In [5]:
batch_size = 1024
num_epochs = 25
show_validation_score_every_epochs = 1
embedding_size = 16
learning_rate = 1e-3

In [6]:
def mse_loss(predictions, target):
    return torch.mean((predictions - target) ** 2)

# Build Dataloaders
train_users_torch = torch.tensor(train_users, device=device)
train_movies_torch = torch.tensor(train_movies, device=device)
train_predictions_torch = torch.tensor(train_predictions, device=device)

train_dataloader = DataLoader(
    TensorDataset(train_users_torch, train_movies_torch, train_predictions_torch),
    batch_size=batch_size)

test_users_torch = torch.tensor(test_users, device=device)
test_movies_torch = torch.tensor(test_movies, device=device)

test_dataloader = DataLoader(
    TensorDataset(test_users_torch, test_movies_torch),
    batch_size=batch_size)

ncf = NCF(number_of_users, number_of_movies, embedding_size).to(device)

optimizer = optim.Adam(ncf.parameters(),
                       lr=learning_rate)

NameError: name 'train_users' is not defined